In [1]:
from __future__ import print_function
import os
from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from models import G, D, weights_init
from data import get_training_set, get_test_set
import torch.backends.cudnn as cudnn
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
#torch.backends.cudnn.enabled = False


In [2]:
import numpy as np

In [3]:

batchSize=10
testBatchSize=10
nEpochs=500000
threads=2
lamb=100
seed=187
cuda=True
beta1=0.5
lr=0.0002
ndf=64
ngf=64
input_nc=3
output_nc=3

dataset='hdr'
dataroot='all_batched_dataset_256/mat_files/'
path=dataroot

import os
import scipy.io
ls=os.listdir(path)
#print ls
xs=[]
ys=[]
#for idx,i in enumerate(ls):
#    if(idx%100==0):
#        print ((idx,'ls'))
#    im1 = scipy.io.loadmat(path+i)
#    im1=im1['im']
#    xs.append(im1.shape[0])
#    ys.append(im1.shape[1])
s1=(256)
s2=(256)
print (s1)
print (s2)
imageSize=(s1,s2,3)

s1x=256
s2x=256
##################
ls=ls[0:100]

#####################
lls=len(ls)

import numpy as np
print (lls*3*s1*s2)
#X=np.zeros((lls,3,s1,s2))
#Y=np.zeros((lls,3,s1,s2))
X=np.random.rand(lls,3,s1,s2)
Y=np.random.rand(lls,3,s1x,s2x)

print ('phew')
count=0
import Image
for idx,i in enumerate(ls):
    li=len(i)
    i=i[0:li-4]
    im1 = scipy.io.loadmat(path+i+'.mat')
    im2=np.asarray(Image.open('all_batched_dataset_256/image_files/'+i+'.png'))    
    im2=im2/255.0;
    im1=im1['im']

    X[count,0,:,:]=im1[:    , : ,0]
    X[count,1,:,:]=im1[ :    , : ,1]
    X[count,2,:,:]=im1[:    , : ,2]

    Y[count,0, :    , : ]=im2[ :    , : ,0]
    Y[count,1, :    , : ]=im2[ :    , : ,1]
    Y[count,2, :    , : ]=im2[ :    , : ,2]
    count=count+1
    if(idx%1000==0):
        print (idx)
print(X.shape)


if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

cudnn.benchmark = True

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)




256
256
19660800
phew
0
(100, 3, 256, 256)


In [4]:

def get_batch_train():
    import numpy.random
    pos=numpy.random.randint(0,X.shape[0],(batchSize))
    x1=X[pos]
    y1=Y[pos]
    x1=torch.FloatTensor(x1)
    y1=torch.FloatTensor(y1)
    return (x1), (y1)


x1,y1=get_batch_train()
print('===> Building model')
netG = G(input_nc, output_nc, ngf)
netG.apply(weights_init)
netD = D(input_nc, output_nc, ndf)
netD.apply(weights_init)



===> Building model


D (
  (conv1): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (batch_norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (batch_norm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (batch_norm8): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (leaky_relu): LeakyReLU (0.2, inplace)
  (sigmoid): Sigmoid ()
)

In [5]:
criterion = nn.BCELoss()
criterion_l1 = nn.L1Loss()
criterion_mse = nn.MSELoss()

real_A = torch.FloatTensor(batchSize, input_nc, s1, s2)
fake_v = torch.FloatTensor(batchSize, input_nc, s1, s2)
real_B = torch.FloatTensor(batchSize, output_nc, s1x, s2x)
label = torch.FloatTensor(batchSize)
real_label = 1
fake_label = 0

if cuda:
    netD = netD.cuda()
    netG = netG.cuda()
    criterion = criterion.cuda()
    criterion_l1 = criterion_l1.cuda()
    criterion_mse = criterion_mse.cuda()
    real_A = real_A.cuda()
    real_B = real_B.cuda()
    fake_v = fake_v.cuda()
    
    label = label.cuda()


real_A = Variable(real_A)
fake_v = Variable(fake_v)
real_B = Variable(real_B)
label = Variable(label)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))



In [6]:

outf='output_dilated_true/'
def train(epoch):
    for iteration in range(10):        
        batchX,batchY=get_batch_train()
        batch=(batchX,batchY)
        print ('ga')
        ############################
        # (1) Update D network: maximize log(D(x,y)) + log(1 - D(x,G(x)))
        ###########################
        # train with real
        netD.volatile = False
        netD.zero_grad()
        real_a_cpu, real_b_cpu = batch[0], batch[1]
        #print(real_a_cpu)
        #print (real_a_cpu)
        #print (type(real_b_cpu))
        #print ((real_A.data))
        
        
        real_A.data.resize_(real_a_cpu.size()).copy_(real_a_cpu)
        real_B.data.resize_(real_b_cpu.size()).copy_(real_b_cpu)

        output = netD(torch.cat(  (real_A, real_B), 1))
        label.data.resize_(output.size()).fill_(real_label)
        err_d_real = criterion(output, label)
        err_d_real.backward()
        d_x_y = output.data.mean()

        # train with fake
        fake_b = netG(real_A)
        '''
        import numpy as np
        xxva2= np.zeros((batchSize,3,s1,s2))
        xxva=(fake_b.data.numpy())
        for va1 in range(batchSize):
            xxva2[va1,:,5:s1-5,5:s1-5]=xxva[va1,:,:,:]
            
        
        print (xxva2.shape)
        
        fake_b=Variable(torch.from_numpy(xxva2   ).float().cuda())
        '''
        
        output = netD(torch.cat((real_A, fake_b.detach()), 1))
        label.data.resize_(output.size()).fill_(fake_label)
        err_d_fake = criterion(output, label)
        err_d_fake.backward()
        d_x_gx = output.data.mean()

        err_d = (err_d_real + err_d_fake) / 2.0
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(x,G(x))) + L1(y,G(x))
        ###########################
        netG.zero_grad()
        netD.volatile = True
        output = netD(torch.cat((real_A, fake_b), 1))
        label.data.resize_(output.size()).fill_(real_label)
        err_g = criterion(output, label) + lamb * criterion_l1(fake_v, real_B)
        err_g.backward()
        d_x_gx_2 = output.data.mean()
        optimizerG.step()

        print("===> Epoch[{}]({}/{}): Loss_D: {:.4f} Loss_G: {:.4f} D(x): {:.4f} D(G(z)): {:.4f}/{:.4f}".format(
            epoch, iteration, 10, err_d.data[0], err_g.data[0], d_x_y, d_x_gx, d_x_gx_2))


        if(iteration==9 and epoch % 5==0):
            import numpy as np
            ss=Y.shape[0]
            xyvap=np.random.randint(ss)
            #vutils.save_image(Y[xyvap],
            #        '%s/real_samples.png' % outf)

            ddv=X[xyvap:xyvap+1]
            #print (ddv.shape)
            ddv2=Variable(torch.from_numpy(ddv).float().cuda())

            ddv3=Y[xyvap:xyvap+1]
            #print (ddv.shape)
            ddv4=Variable(torch.from_numpy(ddv3).float().cuda())
            #print (ddv2)
            fake = netG(ddv2)
            #print (fake.data)
            #print (ddv4)

            vutils.save_image(fake.data,
                    '%s/samples_epoch_%03d.png' % (outf, epoch) , normalize=False)
            vutils.save_image(ddv4.data,
                    '%s/samples_epoch_%03d.png' % (outf, epoch+1), normalize=False)



In [7]:


def test():
    avg_psnr = 0
    for batch in testing_data_loader:
        input, target = Variable(batch[0]), Variable(batch[1])
        if cuda:
            input = input.cuda()
            target = target.cuda()

        prediction = netG(input)
        mse = criterion_mse(prediction, target)
        psnr = 10 * log10(1 / mse.data[0])
        avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))


def checkpoint(epoch):
    if not os.path.exists("checkpoint"):
        os.mkdir("checkpoint")
    if not os.path.exists(os.path.join("checkpoint", dataset)):
        os.mkdir(os.path.join("checkpoint", dataset))
    net_g_model_out_path = "checkpoint/{}/netG_model_epoch_{}.pth".format(dataset, epoch)
    net_d_model_out_path = "checkpoint/{}/netD_model_epoch_{}.pth".format(dataset, epoch)
    torch.save(netG.state_dict(), net_g_model_out_path)
    torch.save(netD.state_dict(), net_d_model_out_path)
    print("Checkpoint saved to {}".format("checkpoint" + dataset))

for epoch in range(0, nEpochs + 1):
    print (epoch)
    train(epoch)
    #test()
    if epoch % 50 == 0:
        checkpoint(epoch)


0
ga
('e1', torch.Size([10, 64, 257, 257]))
('e2', torch.Size([10, 128, 255, 255]))
('e3', torch.Size([10, 256, 251, 251]))
('e3', torch.Size([10, 256, 243, 243]))
('e3', torch.Size([10, 256, 243, 243]))
('e3', torch.Size([10, 256, 243, 243]))
('e3', torch.Size([10, 256, 243, 243]))


RuntimeError: inconsistent tensor sizes at /b/wheel/pytorch-src/torch/lib/THC/generic/THCTensorMath.cu:141